## Preparing Data

In [1]:
##todo 
# 1 分析数据
# 2 复制模型
# 3 插入其中
# 4 运行

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from  torchtext.data import Field
import torchtext
import random
import math
import time

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
tok=lambda x: x.split()
SAMPLE = Field(tokenize=tok,eos_token='<eos>',init_token='<sos>',include_lengths=False)
LABEL = Field(tokenize=tok,eos_token='<eos>',init_token='<sos>',include_lengths=False)

In [5]:
SEED = 1234

random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [6]:
train_sample_path='../data/train/dbn.in.txt'
train_label_path = '../data/train/dbn.out.txt'

vaild_sample_path='../data/vaild/dbn.in.txt'
vaild_label_path = '../data/vaild/dbn.out.txt'



In [7]:
train_sample_list= open(train_sample_path,'r',encoding='utf-8').readlines()
train_label_list = open(train_label_path,'r',encoding='utf-8').readlines()

vaild_sample_list= open(vaild_sample_path,'r',encoding='utf-8').readlines()
vaild_label_list = open(vaild_label_path,'r',encoding='utf-8').readlines()


In [8]:
examples=[]
for x,t in zip(train_sample_list,train_label_list):
    e=torchtext.data.Example()
    setattr(e,'sample',SAMPLE.preprocess(x))
    setattr(e,'label',LABEL.preprocess(t))
    examples.append(e)

In [9]:
vaild_example=[]
for x,t in zip(vaild_sample_list,vaild_label_list):
    e=torchtext.data.Example()
    setattr(e,'sample',SAMPLE.preprocess(x))
    setattr(e,'label',LABEL.preprocess(t))
    vaild_example.append(e)

In [10]:
from collections import Counter

In [11]:
SAMPLE.__dict__.keys()

dict_keys(['sequential', 'use_vocab', 'init_token', 'eos_token', 'unk_token', 'fix_length', 'dtype', 'preprocessing', 'postprocessing', 'lower', 'tokenizer_args', 'tokenize', 'include_lengths', 'batch_first', 'pad_token', 'pad_first', 'truncate_first', 'stop_words', 'is_target'])

In [12]:
c_sample=Counter()
c_label =Counter()
for e in examples:
    c_sample.update(e.sample)
    c_label.update(e.label)

In [13]:
specials=[SAMPLE.unk_token,SAMPLE.pad_token,SAMPLE.init_token,SAMPLE.eos_token]   # 请把unk_token 放第一位，因为源码会直接把unk词映射到0
trg_specials=[SAMPLE.unk_token,SAMPLE.pad_token,SAMPLE.init_token,SAMPLE.eos_token]

In [14]:
SAMPLE.vocab=SAMPLE.vocab_cls(c_sample,specials=specials,min_freq=1000)
LABEL.vocab=LABEL.vocab_cls(c_label,specials=trg_specials)


In [15]:
len(LABEL.vocab)

7

In [16]:
LABEL.vocab.stoi

defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x7f6dfd78d898>>,
            {'<unk>': 0,
             '<pad>': 1,
             '<sos>': 2,
             '<eos>': 3,
             '.': 4,
             ')': 5,
             '(': 6})

In [17]:
train_set=torchtext.data.Dataset(examples,{'sample':SAMPLE,'label':LABEL})
vaild_set=torchtext.data.Dataset(vaild_example,{'sample':SAMPLE,'label':LABEL})

In [18]:
train_set.__dict__.keys()

dict_keys(['examples', 'fields'])

In [19]:
len(vaild_set)

3311

In [20]:
train_iterator=torchtext.data.BucketIterator(train_set,batch_size=32,sort_key=lambda x:len(x.sample),sort_within_batch=True,device=device)
vaild_iterator=torchtext.data.BucketIterator(vaild_set,batch_size=32,sort_key=lambda x:len(x.sample),sort_within_batch=True,device=device)


## Building the Seq2Seq Model

In [21]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src sent len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src sent len, batch size, emb dim]
        
        outputs, (hidden, cell) = self.rnn(embedded)
        
        #outputs = [src sent len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden, cell

In [22]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.out = nn.Linear(hid_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, cell):
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #n directions in the decoder will both always be 1, therefore:
        #hidden = [n layers, batch size, hid dim]
        #context = [n layers, batch size, hid dim]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
                
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        
        #output = [sent len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #sent len and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [n layers, batch size, hid dim]
        #cell = [n layers, batch size, hid dim]
        
        prediction = self.out(output.squeeze(0))
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden, cell

In [51]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.n_layers == decoder.n_layers, \
            "Encoder and decoder must have equal number of layers!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src sent len, batch size]
        #trg = [trg sent len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        batch_size = src.shape[1]
        max_len = src.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(max_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden, cell = self.encoder(src)
        
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, max_len):
            
            #insert input token embedding, previous hidden and previous cell states
            #receive output tensor (predictions) and new hidden and cell states
            output, hidden, cell = self.decoder(input, hidden, cell)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1
        
        return outputs

## Train Model

In [52]:
INPUT_DIM = len(SAMPLE.vocab)
OUTPUT_DIM = len(LABEL.vocab)
ENC_EMB_DIM = 4
DEC_EMB_DIM = 4
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

model = Seq2Seq(enc, dec, device).to(device)
# if torch.cuda.device_count() > 1:
#     print("Let's use", torch.cuda.device_count(), "GPUs!")
#     # dim = 0 [30, xxx] -> [10, ...], [10, ...], [10, ...] on 3 GPUs
#     model = nn.DataParallel(model)
    
model.to(device)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(8, 4)
    (rnn): LSTM(4, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(7, 4)
    (rnn): LSTM(4, 512, num_layers=2, dropout=0.5)
    (out): Linear(in_features=512, out_features=7, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [25]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(8, 4)
    (rnn): LSTM(4, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(7, 4)
    (rnn): LSTM(4, 512, num_layers=2, dropout=0.5)
    (out): Linear(in_features=512, out_features=7, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [26]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 6,327,875 trainable parameters


In [27]:
optimizer = optim.Adam(model.parameters())

In [28]:
PAD_IDX = SAMPLE.vocab.stoi['<pad>']

criterion = nn.CrossEntropyLoss(ignore_index = PAD_IDX)

In [29]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.sample
        trg = batch.label

        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg sent len, batch size]
        #output = [trg sent len, batch size, output dim]
        
        output = output[1:].view(-1, output.shape[-1])
        trg = trg[1:].view(-1)
        
        #trg = [(trg sent len - 1) * batch size]
        #output = [(trg sent len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [30]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.sample
            trg = batch.label
            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg sent len, batch size]
            #output = [trg sent len, batch size, output dim]

            output = output[1:].view(-1, output.shape[-1])
            trg = trg[1:].view(-1)

            #trg = [(trg sent len - 1) * batch size]
            #output = [(trg sent len - 1) * batch size, output dim]

            loss = criterion(output, trg)
            
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [31]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [32]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')
print("start")
for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, vaild_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

start
Epoch: 01 | Time: 2m 29s
	Train Loss: 0.892 | Train PPL:   2.441
	 Val. Loss: 1.467 |  Val. PPL:   4.336
Epoch: 02 | Time: 2m 28s
	Train Loss: 0.761 | Train PPL:   2.141
	 Val. Loss: 1.680 |  Val. PPL:   5.367
Epoch: 03 | Time: 2m 28s
	Train Loss: 0.729 | Train PPL:   2.074
	 Val. Loss: 1.672 |  Val. PPL:   5.322
Epoch: 04 | Time: 2m 28s
	Train Loss: 0.694 | Train PPL:   2.001
	 Val. Loss: 1.536 |  Val. PPL:   4.646
Epoch: 05 | Time: 2m 29s
	Train Loss: 0.660 | Train PPL:   1.936
	 Val. Loss: 1.939 |  Val. PPL:   6.951
Epoch: 06 | Time: 2m 29s
	Train Loss: 0.638 | Train PPL:   1.893
	 Val. Loss: 1.877 |  Val. PPL:   6.537
Epoch: 07 | Time: 2m 28s
	Train Loss: 0.624 | Train PPL:   1.866
	 Val. Loss: 2.014 |  Val. PPL:   7.492
Epoch: 08 | Time: 2m 28s
	Train Loss: 0.605 | Train PPL:   1.830
	 Val. Loss: 1.826 |  Val. PPL:   6.207
Epoch: 09 | Time: 2m 28s
	Train Loss: 0.589 | Train PPL:   1.802
	 Val. Loss: 1.745 |  Val. PPL:   5.729
Epoch: 10 | Time: 2m 28s
	Train Loss: 0.577 | Tra

In [35]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss = evaluate(model, vaild_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 1.468 | Test PPL:   4.340 |


## Inference

In [48]:
def translate_sentence(model, tokenized_sentence):
    model.eval()
    tokenized_sentence = ['<sos>'] + [t.lower() for t in tokenized_sentence] + ['<eos>']
    numericalized = [SAMPLE.vocab.stoi[t] for t in tokenized_sentence] 
    sentence_length = torch.LongTensor([len(numericalized)]).to(device) 
    tensor = torch.LongTensor(numericalized).unsqueeze(1).to(device)
    print(sentence_length)
    translation_tensor_logits = model(tensor, None, 0) 
    translation_tensor = torch.argmax(translation_tensor_logits.squeeze(1), 1)
    translation = [LABEL.vocab.itos[t] for t in translation_tensor]
    translation = translation[1:]
    return translation

In [37]:
def display_attention(sentence, translation, attention):
    
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(111)
    
    attention = attention.squeeze(1).cpu().detach().numpy()
    
    cax = ax.matshow(attention, cmap='bone')
   
    ax.tick_params(labelsize=15)
    ax.set_xticklabels(['']+['<sos>']+[t.lower() for t in sentence]+['<eos>'], 
                       rotation=45)
    ax.set_yticklabels(['']+translation)

    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()
    plt.close()

In [40]:
example_idx = 100

src = vars(train_set.examples[example_idx])['sample']
trg = vars(train_set.examples[example_idx])['label']

print(f'src = {src}')
print(f'trg = {trg}')

src = ['C', 'C', 'A', 'A', 'A', 'A', 'A', 'U', 'A', 'A', 'A', 'A', 'A', 'G', 'A', 'U', 'U', 'C', 'G', 'U', 'C', 'U', 'G', 'A', 'A', 'A', 'C', 'G', 'A', 'A', 'C', 'A', 'A', 'A', 'A', 'A', 'U', 'A', 'A', 'A', 'A', 'U', 'U', 'G', 'G', 'A', 'A', 'A', 'U', 'U', 'U', 'U', 'A', 'G', 'G', 'U', 'U', 'U', 'U', 'G', 'U', 'G', 'A', 'G', 'U', 'G', 'A', 'A', 'A', 'G', 'A', 'U', 'G', 'A', 'A', 'U', 'U', 'A', 'U', 'U', 'U', 'U', 'U', 'A', 'C', 'U', 'G', 'C', 'A', 'C', 'U', 'U', 'G', 'G', 'G', 'U', 'A', 'C', 'C', 'C', 'A', 'A', 'G', 'U', 'G']
trg = ['.', '.', '.', '.', '.', '.', '.', '(', ')', '(', '(', '(', '(', '(', '(', '(', '(', '(', '(', '(', '(', '(', '.', '.', '.', '(', '(', '.', '.', '(', '(', '(', '(', '(', '(', '.', '(', '(', '(', '(', '(', '(', '(', '.', '.', '.', ')', ')', ')', ')', ')', ')', ')', '.', '.', ')', ')', ')', ')', ')', ')', '.', '.', ')', ')', '.', '.', '.', ')', ')', ')', ')', ')', ')', ')', ')', '.', ')', ')', ')', ')', ')', '(', ')', '.', '.', ')', ')', ')', ')', ')', ')', '

In [53]:
translation, attention = translate_sentence(model, src)

print(f'predicted trg = {translation}')

tensor([107], device='cuda:0')


TypeError: 'NoneType' object is not subscriptable